Updated on 2025/03/05

* Now stress at grid points on the surface of the **volume** has been extracted into the CSV files.
* It is still necessary to fill the data into the structured grid points.


The COAT membrane was meshed freely into triangular elements.

The current script will only keep the structured grids which will be used to form output for CNN model.

python enviroment 'gmsh_env' is used.


# Import the libraries

Import the libraries to generate random surface

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.spatial import KDTree
import pandas as pd
import sys

Setting the folder where the extracted csv files were saved

In [2]:
print(f"Indicate the directory where csv files generated by python script 'extract_e11_from_rough_coat.py' were saved")
structured_mesh_csv_folder = 'C:\\Abaqus_Works\\roughness_simulation\\output_free_mesh'
structured_grid_xls_folder = 'C:\\Abaqus_Works\\roughness_simulation\\output_structured_grid'

Indicate the directory where csv files generated by python script 'extract_e11_from_rough_coat.py' were saved


# Set cropped structured grids

* Setting interval of the structured grids
* Setting the coordinates of grids to be mapped according to cropping or sub-dividing

In [ ]:
# Define the interval of structured grid which is denser than the control points
interval_struct_grids = 0.004   #   with unit of 'um'
print(f"The interval of structured grid is {interval_struct_grids} mm.")

# Number of the cropped structured grids along X- or Y-axes
number_struct_grids = 320

original_length_mm = (number_struct_grids - 1)*interval_struct_grids
# Crop the edge of the structured grids for later CNN application
# half_crop_grids = 0

half_crop_grids = 0
print(f"The number grids to be cropped off along the edge is {int(half_crop_grids)}")

sub_times = 2
print(f"The subtime of the meshing is {int(sub_times)}")

number_crop_struct_grids = number_struct_grids - half_crop_grids*2
print(f"The number of the structured grids along X- or Y-axes is {number_crop_struct_grids}")
print(f"The length of the structured grids is {round((number_crop_struct_grids-1)*interval_struct_grids*1000)} um")

# Range of the cropped rough surface model
range_low_crop_struct_grids = interval_struct_grids*half_crop_grids
range_upp_crop_struct_grids = interval_struct_grids*(number_struct_grids-1) - range_low_crop_struct_grids

print(f"the low limit of the cropped grids is {range_low_crop_struct_grids}")
print(f"the upp limit of the cropped grids is {range_upp_crop_struct_grids}")

x_1D_crop_struct_grids = np.linspace(range_low_crop_struct_grids, range_upp_crop_struct_grids, number_crop_struct_grids)
y_1D_crop_struct_grids = x_1D_crop_struct_grids

print(x_1D_crop_struct_grids)

# (X, Y) coordinates of the nodes at the structured grids
nodes_XY_struct_grids = []

for row in range(number_crop_struct_grids):
  for col in range(number_crop_struct_grids):
    node_XY = [x_1D_crop_struct_grids[row], y_1D_crop_struct_grids[col]]
    nodes_XY_struct_grids.append(node_XY)


The interval of structured grid is 0.002 mm.
The number grids to be cropped off along the edge is 0
The subtime of the meshing is 2
The number of the structured grids along X- or Y-axes is 320
The length of the structured grids is 638 um
the low limit of the cropped grids is 0.0
the upp limit of the cropped grids is 0.638
[0.    0.002 0.004 0.006 0.008 0.01  0.012 0.014 0.016 0.018 0.02  0.022
 0.024 0.026 0.028 0.03  0.032 0.034 0.036 0.038 0.04  0.042 0.044 0.046
 0.048 0.05  0.052 0.054 0.056 0.058 0.06  0.062 0.064 0.066 0.068 0.07
 0.072 0.074 0.076 0.078 0.08  0.082 0.084 0.086 0.088 0.09  0.092 0.094
 0.096 0.098 0.1   0.102 0.104 0.106 0.108 0.11  0.112 0.114 0.116 0.118
 0.12  0.122 0.124 0.126 0.128 0.13  0.132 0.134 0.136 0.138 0.14  0.142
 0.144 0.146 0.148 0.15  0.152 0.154 0.156 0.158 0.16  0.162 0.164 0.166
 0.168 0.17  0.172 0.174 0.176 0.178 0.18  0.182 0.184 0.186 0.188 0.19
 0.192 0.194 0.196 0.198 0.2   0.202 0.204 0.206 0.208 0.21  0.212 0.214
 0.216 0.218 0.22  0.

# Extract Grid Value to Excel

* KD tree will be used to search the nodes overlapped with the structured grids.

* Field at the overlapped nodes will be output

In [ ]:
# Read the csv file containing all nodes on the COAT membrane created by 'extract_e11_from_rough_coat.py'.
# id_job_start = 1000
# id_job_end = 1200

id_job_start = 10000
id_job_end = 10001

for id_job in range(id_job_start, id_job_end, 1):
    # csv data file
    # effective_crop_length = (number_struct_grids-1)*interval_struct_grids
    print(f">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    data_csv_file = f"abq_surface_L{round(original_length_mm*1000)}um_C{int(half_crop_grids)}_S{int(sub_times)}_ID{id_job:0{5}}.csv"

    print(f"Current csv file name is {data_csv_file}")
    data_csv_path = os.path.join(structured_mesh_csv_folder, data_csv_file)

    data_all_nodes_string = pd.read_csv(data_csv_path, header=None, skiprows=1)
    # print(data_all_nodes_string.shape)
    # Column-1 is the node label, Column-2:4 are coordinates 
    nodes_pseudo_free_data = data_all_nodes_string.to_numpy(dtype=float)
    # print(nodes_pseudo_free_data.shape)
    nodes_pseudo_free_xy = nodes_pseudo_free_data[:,1:3]
    print(nodes_pseudo_free_xy.shape)
    # create K-D Tree
    tree = KDTree(nodes_pseudo_free_xy)

    # normalized stress at the structured grids
    norm_stress_struct_grids = np.zeros((number_crop_struct_grids, number_crop_struct_grids, 18))

    # register the structured grids to the nodes in the free mesh
    # match_struct_to_free = []
    for idx_struct, node_struct in enumerate(nodes_XY_struct_grids):
        dist, idx_free = tree.query([node_struct])
        if idx_free is None or len(idx_free) == 0 or dist > 0.01:
            print(f"KDTree query failed for node {node_struct}")
        # updated on 2025/04/11
        norm_stress_struct_grids[row, col, :] = nodes_pseudo_free_data[idx_free[0], 4:]
        # match_struct_to_free.append(idx_free[0])

    
    # for row in range(number_crop_struct_grids):
    #     for col in range(number_crop_struct_grids):
    #         row_free_node = match_struct_to_free[row*number_crop_struct_grids+col]
    #         # print(f"row: {row}, col: {col}, row_free_node: {row_free_node}")
    #         # print(len(nodes_pseudo_free_data[row_free_node, 4:]))
    #         norm_stress_struct_grids[row, col, :] = nodes_pseudo_free_data[row_free_node, 4:]

    # write the strain at the structured grids using numpy
    file_struct_grids = f"output_norm_stress_struct_L{round(original_length_mm*1000)}um_C{int(half_crop_grids)}_S{int(sub_times)}_ID{id_job:0{5}}.xlsx"
    file_struct_grids_path = os.path.join(structured_grid_xls_folder, file_struct_grids)

    steps_name = ['TENSILE_X', 'TENSILE_Y', 'SHEAR']
    sum_step = len(steps_name)
    ss_comp = ['11', '22', '33', '12', '13', '23']
    with pd.ExcelWriter(file_struct_grids_path, engine='openpyxl') as writer:
        for i in range(sum_step*6):  
            sheet_name = f"{steps_name[i // 6]}-S{ss_comp[i % 6]}"
            df = pd.DataFrame(norm_stress_struct_grids[:, :, i])
            df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

    print(f"Writing file: {file_struct_grids_path}")
    print(f"<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Current csv file name is abq_surface_L638um_C0_S2_ID10000.csv
(102400, 2)
Writing file: C:\Abaqus_Works\roughness_simulation\output_structured_grid\output_norm_stress_struct_L638um_C0_S2_ID10000.xlsx
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


# Extract R and S at grid points of a square volume with a square hole

In [11]:
print(f"Indicate the directory where csv files generated by python script 'extract_e11_from_rough_coat.py' were saved")

volume_hole_folder = 'C:\\Abaqus_Works\\roughness_simulation\\plate_with_hole'

rough_or_flat = 'flat'
data_csv_file = f"abaqus_hole_{rough_or_flat}.csv"
file_struct_grids = f"output_stress_hole_{rough_or_flat}.xlsx"

steps_name = ['TENSILE_X']

sum_step = len(steps_name)

Indicate the directory where csv files generated by python script 'extract_e11_from_rough_coat.py' were saved


In [12]:
# Define the interval of structured grid which is denser than the control points
interval_struct_grids = 0.004   #   with unit of 'um'
print(f"The interval of structured grid is {interval_struct_grids} mm.")

# Number of the cropped structured grids along X- or Y-axes
number_struct_grids = 301

original_length_mm = (number_struct_grids - 1)*interval_struct_grids
# Crop the edge of the structured grids for later CNN application
# half_crop_grids = 0

half_crop_grids = 0
print(f"The number grids to be cropped off along the edge is {int(half_crop_grids)}")


number_crop_struct_grids = number_struct_grids - half_crop_grids*2
print(f"The number of the structured grids along X- or Y-axes is {number_crop_struct_grids}")
print(f"The length of the structured grids is {round((number_crop_struct_grids-1)*interval_struct_grids*1000)} um")

indx_number_crop_ctrl = (number_crop_struct_grids - 1) // 2

# Range of the cropped rough surface model
range_low_crop_struct_grids = -indx_number_crop_ctrl*interval_struct_grids
range_upp_crop_struct_grids = +indx_number_crop_ctrl*interval_struct_grids

print(f"the low limit of the cropped grids is {range_low_crop_struct_grids}")
print(f"the upp limit of the cropped grids is {range_upp_crop_struct_grids}")

x_1D_crop_struct_grids = np.linspace(range_low_crop_struct_grids, range_upp_crop_struct_grids, number_crop_struct_grids)
y_1D_crop_struct_grids = x_1D_crop_struct_grids

print(x_1D_crop_struct_grids)

half_number_crop_ctrl = 10
low_lim_crop_ctrl = indx_number_crop_ctrl - half_number_crop_ctrl
upp_lim_crop_ctrl = indx_number_crop_ctrl + half_number_crop_ctrl

void_index_surface = np.zeros((number_crop_struct_grids, number_crop_struct_grids), dtype=int)
for i in range(number_crop_struct_grids):
    for j in range(number_crop_struct_grids):
        # if the struct grid point is in the square hole
        if (low_lim_crop_ctrl < i) and (i < upp_lim_crop_ctrl) and (low_lim_crop_ctrl <j) and (j < upp_lim_crop_ctrl):
            # print(f"skip node at {i} and {j}")
            void_index_surface[i, j] = 1


The interval of structured grid is 0.004 mm.
The number grids to be cropped off along the edge is 0
The number of the structured grids along X- or Y-axes is 301
The length of the structured grids is 1200 um
the low limit of the cropped grids is -0.6
the upp limit of the cropped grids is 0.6
[-0.6   -0.596 -0.592 -0.588 -0.584 -0.58  -0.576 -0.572 -0.568 -0.564
 -0.56  -0.556 -0.552 -0.548 -0.544 -0.54  -0.536 -0.532 -0.528 -0.524
 -0.52  -0.516 -0.512 -0.508 -0.504 -0.5   -0.496 -0.492 -0.488 -0.484
 -0.48  -0.476 -0.472 -0.468 -0.464 -0.46  -0.456 -0.452 -0.448 -0.444
 -0.44  -0.436 -0.432 -0.428 -0.424 -0.42  -0.416 -0.412 -0.408 -0.404
 -0.4   -0.396 -0.392 -0.388 -0.384 -0.38  -0.376 -0.372 -0.368 -0.364
 -0.36  -0.356 -0.352 -0.348 -0.344 -0.34  -0.336 -0.332 -0.328 -0.324
 -0.32  -0.316 -0.312 -0.308 -0.304 -0.3   -0.296 -0.292 -0.288 -0.284
 -0.28  -0.276 -0.272 -0.268 -0.264 -0.26  -0.256 -0.252 -0.248 -0.244
 -0.24  -0.236 -0.232 -0.228 -0.224 -0.22  -0.216 -0.212 -0.208 -0.20

In [13]:
# effective_crop_length = (number_struct_grids-1)*interval_struct_grids
print(f">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")


print(f"Current csv file name is {data_csv_file}")
data_csv_path = os.path.join(volume_hole_folder, data_csv_file)

data_all_nodes_string = pd.read_csv(data_csv_path, header=None, skiprows=1)
# print(data_all_nodes_string.shape)
# Column-1 is the node label, Column-2:4 are coordinates 
nodes_pseudo_free_data = data_all_nodes_string.to_numpy(dtype=float)
# print(nodes_pseudo_free_data.shape)
nodes_pseudo_free_xy = nodes_pseudo_free_data[:,1:3]
print(nodes_pseudo_free_xy.shape)
# create K-D Tree
tree = KDTree(nodes_pseudo_free_xy)

# (X, Y) coordinates of the nodes at the structured grids
nodes_XY_struct_grids = []

norm_stress_struct_grids = np.zeros((number_crop_struct_grids, number_crop_struct_grids, 6*sum_step))

for row in range(number_crop_struct_grids):
  for col in range(number_crop_struct_grids):

    if void_index_surface[row, col] == 1:
        # skip void
        continue

    node_XY = [x_1D_crop_struct_grids[row], y_1D_crop_struct_grids[col]]
    dist, idx_free = tree.query([node_XY])
    if idx_free is None or len(idx_free) == 0 or dist > 0.01:
        print(f"KDTree query failed for node row={row}, col={col}")
        continue

    norm_stress_struct_grids[row, col, :] = nodes_pseudo_free_data[idx_free[0], 4:]

# write the strain at the structured grids using numpy
file_struct_grids_path = os.path.join(volume_hole_folder, file_struct_grids)

ss_comp = ['11', '22', '33', '12', '13', '23']
with pd.ExcelWriter(file_struct_grids_path, engine='openpyxl') as writer:
    for i in range(sum_step*6):  
        sheet_name = f"{steps_name[i // 6]}-S{ss_comp[i % 6]}"
        df = pd.DataFrame(norm_stress_struct_grids[:, :, i])
        df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

print(f"Writing file: {file_struct_grids_path}")
print(f"<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Current csv file name is abaqus_hole_flat.csv
(90240, 2)
Writing file: C:\Abaqus_Works\roughness_simulation\plate_with_hole\output_stress_hole_flat.xlsx
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
